In [2]:
import os
import re
import pandas as pd

# Extracción

Primero .txt

In [8]:
def procesarOracion(bloque):
    #Eliminar punttuación y pasar a minúsculas
    bloque = re.sub(r'[^\w\s]', '', bloque)
    bloque = bloque.lower()
    return bloque

In [14]:
#Por cada archivo en la carpeta textos, contar oraciones
data = {}
primera = True
id = locutor = texto = palabras = traduccion = morfologia = pos = None

for filename in os.listdir('../textos'):
    #Si el archivo es un .txt
    if filename.endswith('.txt'):
        #Se abre el archivo
        #Saltar el archivo 'textos/DICCIONARIOISKONAWA7.txt' porque no es un archivo de texto
        if filename == 'DICCIONARIOISKONAWA7.txt':
            continue
        with open('../textos/'+filename, encoding='utf-8') as file:
            #Se lee el archivo
            texto = file.read()
            #Por cada linea en el archivo
            for line in texto.split('\n'):
                #Skip si la linea es solo espacios en blanco o saltos de linea
                if not line.strip():
                    continue
                tipo_match = re.search(r'^\\\S*', line)
                if tipo_match:
                    tipo = tipo_match.group()
                    # Remove the tipo from the line to get the oracion
                    oracion = re.sub(r'^\\\S* ', '', line)
                    #Quitar espacios en blnaco
                    oracion = oracion.strip()
                    
                    # Process the oracion based on the tipo
                    if tipo == '\\ref':
                        id = oracion
                        # Append the previous data before resetting variables
                        if not primera:
                            key = f"{filename}_{id}"
                            data[key] = {
                                'id': id, 
                                'locutor': locutor, 
                                'texto': texto, 
                                'palabras': palabras, 
                                'traduccion': traduccion, 
                                'morfologia': morfologia, 
                                'pos': pos,
                                'archivo': filename
                            }
                        primera = False
                    elif tipo == '\\ELANParticipant':
                        locutor = oracion
                    elif tipo == '\\trs':
                        texto = procesarOracion(oracion)
                    elif tipo == '\\tx':
                        palabras = oracion
                    elif tipo == '\\ft':
                        traduccion = oracion
                    elif tipo == '\\mb':
                        morfologia = oracion
                    elif tipo == '\\ps':
                        pos = oracion
                    else:
                        continue
            primera = True
if not primera:
    key = f"{filename}_{id}"
    data[key] = {
        'id': id, 
        'locutor': locutor, 
        'texto': texto, 
        'palabras': palabras, 
        'traduccion': traduccion, 
        'morfologia': morfologia, 
        'pos': pos,
        'archivo': filename
    }

df = pd.DataFrame(data)                

Ahora solo los .eaf 

In [15]:
#Los archivos que terminan en .eaf y no tienen un archivo .txt
archivos = []
for filename in os.listdir('../textos'):
    if filename.endswith('.eaf'):
        if not os.path.exists('../textos/'+filename[:-4]+'.txt'):
            archivos.append(filename)

In [55]:
def leer_eaf(filename, data):
    locutor = texto = palabras = traduccion = morfologia = pos = id = None
    dentro_tier = False
    with open('../textos/'+filename, encoding='utf-8') as file:
        texto = file.read()
        annotation_id = None
        #Cada archivo eaf es un xml. Nos interesa el valor de los atributos 'ANNOTATION_VALUE' de los elementos 'TIER' que tengan el atributo 'TIER_ID' igual a 'trsx@algo'
        #Por cada linea en el archivo
        for line in texto.split('\n'):
            #Skip si la linea es solo espacios en blanco o saltos de linea
            if not line.strip():
                continue
            #Buscar fin de tier
            end_tier_match = re.search(r'</TIER>', line)
            if end_tier_match:
                # print('End tier')
                dentro_tier = False

            if dentro_tier:
                if tier_id.startswith('trs@'):
                    #Cosa de transcripción original
                    alignable_annotation_match = re.search(r'<ALIGNABLE_ANNOTATION[^>]*ANNOTATION_ID="([^"]+)"', line)
                    if alignable_annotation_match:
                            annotation_id = alignable_annotation_match.group(1)
                            id = annotation_id
                    if id is not None:
                        annotation_value_match = re.search(r'<ANNOTATION_VALUE>([^<]+)</ANNOTATION_VALUE>', line)
                        if annotation_value_match:
                            texto = procesarOracion(annotation_value_match.group(1))
                            key = f"{filename}_{id}"
                            data[key] = {
                                'id': id, 
                                'locutor': locutor, 
                                'texto': texto, 
                                'palabras': palabras, 
                                'traduccion': traduccion, 
                                'morfologia': morfologia, 
                                'pos': pos,
                                'archivo': filename
                            }
                            annotation_id = None

                elif tier_id.startswith('tx@'):
                    ref_annotation_match = re.search(r'<REF_ANNOTATION[^>]*ANNOTATION_ID="([^"]+)" ANNOTATION_REF="([^"]+)"', line)
                    # print("Buscando referencia a anotación en tier tx")
                    if ref_annotation_match:
                        # print ("Se encontró una referencia a una anotación con valor {} y referencia {}".format(ref_annotation_match.group(1), ref_annotation_match.group(2)))
                        annotation_id = ref_annotation_match.group(1)
                        #Buscar en data el elemento con el id igual a annotation_ref y archivo igual a filename
                        annotation_ref = ref_annotation_match.group(2)
                        key = f"{filename}_{annotation_ref}"
                        # print(key)
                    if annotation_id is not None:
                        annotation_value_match = re.search(r'<ANNOTATION_VALUE>([^<]+)</ANNOTATION_VALUE>', line)
                        if annotation_value_match:
                            palabras = annotation_value_match.group(1)
                            data[key]['palabras'] = palabras
                            # print(key)
                            annotation_id = None
                elif tier_id.startswith('ft@'):
                    #Cosa de traducción
                    ref_annotation_match = re.search(r'<REF_ANNOTATION[^>]*ANNOTATION_ID="([^"]+)" ANNOTATION_REF="([^"]+)"', line)
                    if ref_annotation_match:
                        annotation_id = ref_annotation_match.group(1)
                        #Buscar en data el elemento con el id igual a annotation_ref y archivo igual a filename
                        annotation_ref = ref_annotation_match.group(2)
                        key = f"{filename}_{annotation_ref}"
                        # print(key)
                    if annotation_id is not None:
                        annotation_value_match = re.search(r'<ANNOTATION_VALUE>([^<]+)</ANNOTATION_VALUE>', line)
                        if annotation_value_match:
                            traduccion = annotation_value_match.group(1)
                            # print(key)
                            data[key]['traduccion'] = traduccion
                            annotation_id = None
            else:
                #Buscamos primero un elemento 'TIER' que tenga el atributo 'TIER_ID' igual a 'trs@algo'
                tier_match = re.search(r'<TIER[^>]*PARTICIPANT="([^"]+)"[^>]*TIER_ID="([^"]+)"', line)
                if tier_match:
                    locutor = tier_match.group(1)
                    tier_id = tier_match.group(2)
                    # print("Entrando en tier {} con locutor {} en archivo {}".format(tier_id, locutor, filename))
                    dentro_tier = True
            

In [56]:
data = {}

In [57]:
for archivo in archivos:
    leer_eaf(archivo, data)

In [58]:
df = pd.DataFrame(data).transpose().reset_index(drop=True)
df

,id,locutor,texto,palabras,traduccion,morfologia,pos,archivo
0,a42,Juanita,isan koro wistori isan koro wistori,"isan koro wistori, isan koro wistori","(pene) pequeño como ungurahui cenizo, (pene) p...",None,None,JC.NC.IC-cancion.pene.pequeno-2013.eaf
1,a43,Juanita,epe oa katsari epe oa katsari,"epe oa katsari, epe oa katsari","apestoso como la flor de yarina, apestoso como...",None,None,JC.NC.IC-cancion.pene.pequeno-2013.eaf
2,a44,Juanita,wa kin oa katsari wa kin oa katsari,"wa kin oa katsari, wa kin oa katsari",maloliente como la (sobra) de la flor de papay...,None,None,JC.NC.IC-cancion.pene.pequeno-2013.eaf
3,a45,Juanita,isan koro wistori isan koro wistori,"isan koro wistori, isan koro wistori","(pene) pequeño como ungurahui cenizo, (pene) p...",None,None,JC.NC.IC-cancion.pene.pequeno-2013.eaf
4,a46,Juanita,epe oa katsari epe oa katsari,"epe oa katsari, epe oa katsari","maloliente como la flor de yarina, maloliente ...",None,None,JC.NC.IC-cancion.pene.pequeno-2013.eaf
...,...,...,...,...,...,...,...,...
625,a101,Pablo,wetsa nomiroka,Otro también había,otro nombre,None,None,PC-cuento.yuca-2013.eaf
626,a102,Pablo,hawebachi kari hawebachi kari kan,El camote hawebachi.,"el camote ""hawebachi"" (otro tipo de camote, ig...",None,None,PC-cuento.yuca-2013.eaf
627,a103,Pablo,ha hakonshamaribiriki neskashamaribi ha haweba...,"Ese era muy bueno también. Así, ese camote haw...","ese es muy bueno, también; así, también, ese c...",None,None,PC-cuento.yuca-2013.eaf
628,a104,Pablo,pia noeshama noe batashama hawebachi kari,Lo comíamos y era sabroso. El camote rojo era ...,"comíamos, muy sabroso, el camote ""hawebachi"" e...",None,None,PC-cuento.yuca-2013.eaf


## Limpiar texto

In [52]:
from langdetect import detect
import pandas as pd

def is_spanish(text):
    try:
        return detect(text) == 'es'
    except:
        return False

#Quitar de df las filas que están en df_spanish
df = df[~df['texto'].apply(is_spanish)]

In [54]:
import nltk 
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

#Crear un corpus con los textos en df
corpus = PlaintextCorpusReader('../corpus', '.*')
for i, row in df.iterrows():
    #Crear un archivo con el id como nombre y el texto como contenido
    with open(f'../corpus/{row["id"]}.txt', 'w') as file:
        file.write(row['texto'])